# 闭包里变量作用域

作用域是程序运行时变量可被访问的范围，定义在函数内的变量是局部变量，局部变量的作用范围只能是函数内部范围内，它不能在函数外引用。现在我们来考虑如下的例子

In [4]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"
    def printer():
        # printer 是嵌套函数
        print(msg)
    return printer

another = print_msg()
# 输出 zen of python
another()

zen of python


一般情况下，函数中的局部变量仅在函数的执行期间可用，一旦 print_msg() 执行过后，我们会认为 msg变量将不再可用。然而，在这里我们发现 print_msg 执行完之后，在调用 another 的时候 msg 变量的值正常输出了，这就是闭包的作用，闭包使得局部变量在函数外被访问成为可能。

# 使用闭包的作用

作用1：当闭包执行完后，仍然能够保持住当前的运行环境。

作用2：闭包可以根据外部作用域的局部变量来得到不同的结果。

作用3：闭包对数据的持久化以及按配置产生不同的功能，是很有帮助的


闭包主要是在函数式开发过程中使用。 

In [30]:
def adder(x):
    def wrapper(y):
        return x + y
    return wrapper

adder5 = adder(5)
# 输出 15
adder5(10)
# 输出 11
adder5(6)

11

In [31]:
print(adder5.__name__)

wrapper


当闭包执行完后，仍然能够保持住当前的运行环境

举例： 闭包可以保存当前的运行环境，以一个类似棋盘游戏的例子来说明。假设棋盘大小为50*50，左上角为坐标系原点(0,0)，我需要一个函数，接收2个参数，分别为方向(direction)，步长(step)，该函数控制棋子的运动。 这里需要说明的是，每次运动的起点都是上次运动结束的终点。

In [47]:
origin = [0, 0] 
legal_x = [0, 50]  
legal_y = [0, 50] 
def create(pos=origin):   
    def player(direction,step):    
        # 这里应该首先判断参数direction,step的合法性，比如direction不能斜着走，step不能为负等    
        # 然后还要对新生成的x，y坐标的合法性进行判断处理，这里主要是想介绍闭包，就不详细写了。    
        new_x = pos[0] + direction[0]*step    
        new_y = pos[1] + direction[1]*step    
        pos[0] = new_x    
        pos[1] = new_y    
        #注意！此处不能写成 pos = [new_x, new_y]，因为参数变量不能被修改，而pos[]是容器类的解决方法 
        return pos   
    return player    

player = create() # 创建棋子player，起点为原点  
print(player([1,0],10)) # 向x轴正方向移动10步  
print(player([0,1],20)) # 向y轴正方向移动20步  
print(player([-1,0],10)) # 向x轴负方向移动10步 

[10, 0]
[10, 20]
[0, 20]


闭包可以根据外部作用域的局部变量来得到不同的结果

这有点像一种类似配置功能的作用，我们可以修改外部的变量，闭包根据这个变量展现出不同的功能。比如有时我们需要对某些文件的特殊行进行分析，先要提取出这些特殊行。

In [48]:
def make_filter(keep):   
    def the_filter(file_name):    
        file = open(file_name)    
        lines = file.readlines()    
        file.close()    
        filter_doc = [i for i in lines if keep in i]    
        return filter_doc   
    return the_filter 


如果我们需要取得文件”result.txt”中含有”pass”关键字的行，则可以这样使用例子程序

In [ ]:
filter = make_filter("pass") 
filter_result = filter("result.txt") 

# 闭包的_closure_属性

一个函数和它的环境变量合在一起，就构成了一个闭包(closure)。在Python中，所谓的闭包是一个包含有环境变量取值的函数对象。环境变量取值被保存在函数对象的_closure_属性中。


所有函数都有一个 __closure__属性，如果这个函数是一个闭包的话，那么它返回的是一个由 cell 对象 组成的元组对象。cell 对象的cell_contents 属性就是闭包中的自由变量。

In [7]:
adder.__closure__

In [8]:
adder5.__closure__
# (<cell at 0x103075910: int object at 0x7fd251604518>,)

(<cell at 0x00000157BB852078: int object at 0x00007FFA168093C0>,)

In [9]:
adder5.__closure__[0].cell_contents
# 5

5

# 使用闭包注意事项


1 闭包中是不能修改外部作用域局部变量的值

In [34]:
def func():
    m = 0

    def func_in():
        m = 1
        print("++%d" % m)

    print("--%d" % m)
    func_in()
    print("==%d" % m)


print("最后打印:%s" % func())


--0
++1
==0
最后打印:None


2 局部变量的问题

In [36]:
def foo():
    a = 1
    def bar():
        a = a + 1
        return a

    return bar
f=foo()
print(f())

UnboundLocalError: local variable 'a' referenced before assignment

这是因为在执行代码 f = foo()时，Python会导入全部的闭包函数体bar()来分析其的局部变量。Python规则指定所有在赋值语句左面的变量都是局部变量，则在闭包bar()中，变量a在赋值符号”=”的左面，被Python认为是bar()中的局部变量。再接下来执行print(f())时，程序运行至a = a + 1时，因为先前已经把a归为bar()中的局部变量，所以python会在bar()中去找在赋值语句右面的a的值，结果找不到，就会报错。
两种解决办法

In [44]:
#第一种解决办法：自由变量为不可变对象
# python3通过nonlocal关键字来解决，该语句显式的指定a不是闭包的局部变量
def foo():
    a = 1

    def bar():
        nonlocal a
        a = a + 1
        return a

    return bar
f=foo()
print(f())

2


In [45]:
#第二种解决办法：不是太好，不建议，自由变量为可变对象
# 在python3之前没有直接的解决方法，只能间接地通过容器类型来解决，
# 因为容器类型不是存放在栈空间的，inner函数可以访问到。
def foo():
    a = [1]
    def bar():
        a[0] = a[0] + 1
        return a[0]

    return bar
f=foo()
print(f())

2


3 Python函数式编程中一个问题

In [40]:
for a in range(10): 
    print(a)

0
1
2
3
4
5
6
7
8
9


在程序里面经常会出现这类的循环语句，Python的问题就在于：当循环结束以后，循环体中的临时变量a不会销毁，而是继续存在于执行环境中。

还有一个Python的现象是:Python的函数只有在执行时，才会去找函数体里的变量的值。

In [41]:
listname = []
for a in range(3):
    print("a==%s" % a)
    def foo(x):
        print("a--%s" % a)
        print(x + a)
        print("a++%s" % a)
    listname .append(foo)
for f in listname :
    f(2)


a==0
a==1
a==2
a--2
4
a++2
a--2
4
a++2
a--2
4
a++2


可能有些人认为这段代码的执行结果应该是2,3,4.但是实际的结果是4,4,4。这是因为当把函数加入flist列表里时，Python还没有给a赋值，只有当执行时，再去找a的值是什么。这时在for循环语句中，已经将a的值赋值为2，所以以上代码的执行结果是4,4,4。

如果要想结果是2，3，4，看如下代码修改：

In [43]:
listname = []
for a in range(3):
    print("a==%s" % a)
    def foo(x, y=a):
        print("a==%s" % a)
        print("y--%s" % y)
        print(x + y)
        print("y++%s" % y)
        print("a==%s" % a)
    listname.append(foo)
for f in listname:
    f(2)

a==0
a==1
a==2
a==2
y--0
2
y++0
a==2
a==2
y--1
3
y++1
a==2
a==2
y--2
4
y++2
a==2
